<a href="https://colab.research.google.com/github/Smart-Pigs/Cow_Food_Consumption/blob/main/Data_Cleaning%2BDataset%2BML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
#Change path
import os
path = '/content/gdrive/MyDrive/Cow_eating_label/'
os.chdir(path)

#concatenate for outliers detection 

In [5]:
import glob
import numpy as np
import pandas as pd

#remove frist two columns of counts
df = pd.read_csv('Cow6065/Final/uncleaned/cow6065_1pm_to_5pm_1.csv')
column_names = df.columns.tolist()[2:]
print(column_names)

#prepare dataset
folder = 'Cow6065/Final/uncleaned'
output_df='Cow6065/Final/concatenated'
df = pd.DataFrame()
for file in os.listdir(folder):
  if file.endswith(".csv"):
    file_path = os.path.join(folder, file)
    file_df = pd.read_csv(file_path)
    file_df=file_df[column_names]
    file_df["label"] = file_df["label"].replace("grazing", "eating")
    file_df["label"] = file_df["label"].fillna("not_eating")
    file_df["label"] = file_df["label"].replace("ruminating", "not_eating")
    df = pd.concat([df, file_df], ignore_index=True)
    print(f'Add {file}')
output = os.path.join(output_df, '6065_all.csv')
df.to_csv(output, index=False)
    


['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg', 'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar', 'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose', 'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose', 'gyro_z_nose', 'label']
Add cow6065_1pm_to_5pm_1.csv


<ipython-input-5-2ab3875241c6>:17: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Add cow6065_5pm_to_9pm.csv
Add cow6065_9am_to_1pm_1.csv


<ipython-input-5-2ab3875241c6>:17: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Add cow6065_9am_to_1pm_2.csv
Add cow6065_1pm_to_5pm_2.csv
Add cow6065_9pm_to_12am.csv


In [6]:
# 1.5IQR
def Detect_outlier_IQR(df):
  Q1 = df.quantile(0.25)
  Q3 = df.quantile(0.75)
  IQR = Q3 - Q1
  down_threshold = Q1-1.5*IQR
  up_threshold = Q3+1.5*IQR
  return up_threshold,down_threshold


#label set
labels=set(df['label'])
df=pd.read_csv('Cow6065/Final/concatenated/6065_all.csv')
ouliers=0
out_percent=0
for label in labels:
  data_item = df[df['label'] == label]
  print(f"Label: {label}")
  for col in data_item.columns[:-1]:
    col_threshold = Detect_outlier_IQR(data_item[col])
    outliers = data_item[(data_item[col]<col_threshold[1])|(data_item[col]>col_threshold[0])].shape[0]
    print(f"{col}: {col_threshold}")
  print("\n")
out_percent=outliers/df.shape[0] 
print(out_percent) 



Label: not_eating
acc_x_leg: (2.3417358398437504, -3.4913696289062504)
acc_y_leg: (-9.56732177734375, -10.088903808593749)
acc_z_leg: (0.8553466796874999, -2.1305908203125)
gyro_x_leg: (0.9603658765554428, -1.1128049045801163)
gyro_y_leg: (1.0365853458642957, -1.6463414281606672)
gyro_z_leg: (2.2713415175676346, -1.509146377444267)
acc_x_maxilar: (12.182409667968752, -2.36925048828125)
acc_y_maxilar: (11.636303710937497, 1.8650146484375023)
acc_z_maxilar: (-1.9296142578125006, -7.7770751953125)
gyro_x_maxilar: (48.10975646972656, -47.5)
gyro_y_maxilar: (27.2560977935791, -26.158536911010742)
gyro_z_maxilar: (25.594512701034546, -25.990854501724243)
acc_x_nose: (7.88653564453125, 0.12022705078124973)
acc_y_nose: (-3.5428100585937514, -12.409704589843749)
acc_z_nose: (9.110339355468751, -2.2448364257812505)
gyro_x_nose: (31.615854263305664, -33.018293380737305)
gyro_y_nose: (32.515244245529175, -33.94817090034485)
gyro_z_nose: (26.097561597824097, -26.58536648750305)


Label: eating
acc_

#Back to original files and remove outliers

In [7]:
#Label: eating
thres_eat={
    'acc_x_leg': (2.14434814453125, -2.52117919921875),
    'acc_y_leg': (-9.695922851562502, -10.021313476562499),
    'acc_z_leg': (0.08732910156249984, -1.8171630859375),
    'gyro_x_leg': (0.8079268485307691, -1.0213414877653122),
    'gyro_y_leg': (0.9146341681480408, -1.5243902802467346),
    'gyro_z_leg': (2.14939022064209, -1.3871951103210447),
    'acc_x_maxilar': (12.22906494140625, 3.6109985351562517),
    'acc_y_maxilar': (9.8861328125, -2.6318359374999996),
    'acc_z_maxilar': (0.11962890624999956, -8.67548828125),
    'gyro_x_maxilar': (70.67073059082031, -70.06097412109375),
    'gyro_y_maxilar': (41.15853834152222, -39.81707429885864),
    'gyro_z_maxilar': (49.771342277526855, -52.05792713165283),
    'acc_x_nose': (9.349597167968751, -2.5797973632812505),
    'acc_y_nose': (-2.9895263671875023, -13.765698242187497),
    'acc_z_nose': (10.08890380859375, -5.07525634765625),
    'gyro_x_nose': (59.71036624908447, -58.70426845550537),
    'gyro_y_nose': (44.1463418006897, -45.365853786468506),
    'gyro_z_nose': (35.21341514587402, -36.0060977935791)
}


#Label: not_eating
thres_neat = {
    'acc_x_leg': (2.3417358398437504, -3.4913696289062504),
    'acc_y_leg': (-9.56732177734375, -10.088903808593749),
    'acc_z_leg': (0.8553466796874999, -2.1305908203125),
    'gyro_x_leg': (0.9603658765554428, -1.1128049045801163),
    'gyro_y_leg': (1.0365853458642957, -1.6463414281606672),
    'gyro_z_leg': (2.2713415175676346, -1.509146377444267),
    'acc_x_maxilar': (12.182409667968752, -2.36925048828125),
    'acc_y_maxilar': (11.636303710937497, 1.8650146484375023),
    'acc_z_maxilar': (-1.9296142578125006, -7.7770751953125),
    'gyro_x_maxilar': (48.10975646972656, -47.5),
    'gyro_y_maxilar': (27.2560977935791, -26.158536911010742),
    'gyro_z_maxilar': (25.594512701034546, -25.990854501724243),
    'acc_x_nose': (7.88653564453125, 0.12022705078124973),
    'acc_y_nose': (-3.5428100585937514, -12.409704589843749),
    'acc_z_nose': (9.110339355468751, -2.2448364257812505),
    'gyro_x_nose': (31.615854263305664, -33.018293380737305),
    'gyro_y_nose': (32.515244245529175, -33.94817090034485),
    'gyro_z_nose': (26.097561597824097, -26.58536648750305)
}




In [8]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import glob
import numpy as np
import pandas as pd

Input_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/uncleaned'
Output_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/'

df = pd.DataFrame()
for file_name in os.listdir(Input_path):
  if file_name.endswith('.csv'):
    file_path = os.path.join(Input_path, file_name)
    df = pd.read_csv(file_path)
    df = df[df.columns.tolist()[2:]]
    df["label"] = df["label"].replace("grazing", "eating")
    df["label"] = df["label"].fillna("not_eating")
    df["label"] = df["label"].replace("ruminating", "not_eating")
    for label, threshold in [('eating', thres_eat), ('not_eating', thres_neat)]:
      mask = df['label'] == label
      for col in df.columns[:-1]: 
        if col in threshold:
          # Remove outliers
          upper_thresh, lower_thresh = threshold[col]
          outside_thresh_mask = (df[col] < lower_thresh) | (df[col] > upper_thresh)
          df.loc[outside_thresh_mask & mask, col] = np.nan
          # Interpolate  
          df.loc[mask, col] = df.loc[mask, col].interpolate(method='linear', limit_direction='both')
          # df_new = df_new.append(df)
    clean_file_path = os.path.join(Output_path, f'{file_name}')
    df.to_csv(clean_file_path, index=False)
    print(f'Saved {clean_file_path}')
    df = pd.DataFrame()









Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_5pm_to_9pm.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9pm_to_12am.csv


# Block wised

In [12]:
df1 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_1.csv')
df2 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_2.csv')
df3 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_5pm_to_9pm.csv')
df4 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_1.csv')
df5 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_2.csv')
df6 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9pm_to_12am.csv')

In [14]:
data_item.columns[:-1]

Index(['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose'],
      dtype='object')

# Test cow6065_1pm_to_5pm_1


In [50]:
dftest = df1
dftrain = pd.concat([df2,df3,df4,df5,df6], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [51]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(727832, 18)
(206059, 18)
(727832,)
(206059,)


In [52]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(727832, 19)
(206059, 19)


In [53]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [54]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [55]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [56]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [57]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [58]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

5686 5686
1609 1609


In [59]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [60]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [61]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(5686, 162)
(1609, 162)


In [62]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [63]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_1pm_to_5pm_1.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_1pm_to_5pm_1.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_1pm_to_5pm_1.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_1pm_to_5pm_1.csv", y_np_test, delimiter=",",fmt='%s')
     

In [64]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.96      0.98      0.97       588
  not_eating       0.99      0.97      0.98      1021

    accuracy                           0.98      1609
   macro avg       0.97      0.98      0.97      1609
weighted avg       0.98      0.98      0.98      1609



# Test cow6065_1pm_to_5pm_2


In [35]:
dftest = df2
dftrain = pd.concat([df1,df3,df4,df5,df6], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [36]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(862887, 18)
(71004, 18)
(862887,)
(71004,)


In [37]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(862887, 19)
(71004, 19)


In [38]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [39]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [40]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [41]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [42]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [43]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

6741 6741
554 554


In [44]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [45]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [46]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(6741, 162)
(554, 162)


In [47]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [48]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_1pm_to_5pm_2.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_1pm_to_5pm_2.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_1pm_to_5pm_2.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_1pm_to_5pm_2.csv", y_np_test, delimiter=",",fmt='%s')
     

In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.88      0.98      0.93       207
  not_eating       0.99      0.92      0.95       347

    accuracy                           0.94       554
   macro avg       0.94      0.95      0.94       554
weighted avg       0.95      0.94      0.94       554



# Test cow6065_5pm_to_9pm


In [65]:
dftest = df3
dftrain = pd.concat([df1,df2,df4,df5,df6], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [66]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(756615, 18)
(177276, 18)
(756615,)
(177276,)


In [67]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(756615, 19)
(177276, 19)


In [68]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [69]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [70]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [71]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [72]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [73]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

5911 5911
1384 1384


In [74]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [75]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [76]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(5911, 162)
(1384, 162)


In [77]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [78]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_5pm_to_9pm.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_5pm_to_9pm.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_5pm_to_9pm.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_5pm_to_9pm.csv", y_np_test, delimiter=",",fmt='%s')
     

In [79]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.95      1.00      0.97       456
  not_eating       1.00      0.97      0.99       928

    accuracy                           0.98      1384
   macro avg       0.97      0.98      0.98      1384
weighted avg       0.98      0.98      0.98      1384



# Test cow6065_9am_to_1pm_1


In [80]:
dftest = df4
dftrain = pd.concat([df1,df2,df3,df5,df6], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [81]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(881885, 18)
(52006, 18)
(881885,)
(52006,)


In [82]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(881885, 19)
(52006, 19)


In [83]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [84]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [85]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [86]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [87]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [88]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

6889 6889
406 406


In [89]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [90]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [91]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(6889, 162)
(406, 162)


In [92]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [93]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_9am_to_1pm_1.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_9am_to_1pm_1.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_9am_to_1pm_1.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_9am_to_1pm_1.csv", y_np_test, delimiter=",",fmt='%s')
     

In [94]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.98      0.98      0.98       134
  not_eating       0.99      0.99      0.99       272

    accuracy                           0.99       406
   macro avg       0.98      0.98      0.98       406
weighted avg       0.99      0.99      0.99       406



# Test cow6065_9am_to_1pm_2




In [95]:
dftest = df5
dftrain = pd.concat([df1,df2,df3,df4,df6], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [96]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(680163, 18)
(253728, 18)
(680163,)
(253728,)


In [97]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(680163, 19)
(253728, 19)


In [98]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [99]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [100]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [101]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [102]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [103]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

5313 5313
1982 1982


In [104]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [105]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [106]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(5313, 162)
(1982, 162)


In [107]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [108]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_9am_to_1pm_2.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_9am_to_1pm_2.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_9am_to_1pm_2.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_9am_to_1pm_2.csv", y_np_test, delimiter=",",fmt='%s')
     

In [109]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.95      0.99      0.97       920
  not_eating       0.99      0.96      0.98      1062

    accuracy                           0.97      1982
   macro avg       0.97      0.98      0.97      1982
weighted avg       0.98      0.97      0.97      1982



# Test cow6065_9pm_to_12am




In [110]:
dftest = df6
dftrain = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [111]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(760073, 18)
(173818, 18)
(760073,)
(173818,)


In [112]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(760073, 19)
(173818, 19)


In [113]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [114]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [115]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [116]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr
     


In [117]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [118]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

5938 5938
1357 1357


In [119]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [120]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [121]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(5938, 162)
(1357, 162)


In [122]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [123]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_cow6065_9pm_to_12am.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_cow6065_9pm_to_12am.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_cow6065_9pm_to_12am.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_test_cow6065_9pm_to_12am.csv", y_np_test, delimiter=",",fmt='%s')
     

In [124]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.95      1.00      0.97       605
  not_eating       1.00      0.96      0.98       752

    accuracy                           0.98      1357
   macro avg       0.97      0.98      0.98      1357
weighted avg       0.98      0.98      0.98      1357

